In [16]:
# !pip install datasets
# !pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
# import nltk
# from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
# from datasets import Dataset
# from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import torch
# from tokenizers import (
#     decoders,
#     models,
#     normalizers,
#     pre_tokenizers,
#     processors,
#     trainers,
#     Tokenizer,
# )
import numpy as np
from scipy.sparse import save_npz, load_npz

In [2]:
LOWERCASE = True
VOCAB_SIZE = 32000

In [3]:
train = pd.read_csv("data/train_preprocessed_fill_missing.csv")
test = pd.read_csv("data/test_preprocessed_fill_missing.csv")


In [4]:
# train['text'] = train['title'].astype(str) + " " + train['content'].astype(str)
# test['text'] = test['title'].astype(str) + " " + test['content'].astype(str)

In [5]:
# df = pd.concat([train['text'], test['text']], axis=0)

In [6]:
# raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
# raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFKD()] + [normalizers.Lowercase()] if LOWERCASE else [])
# raw_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]", "[BOS]"]
# trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

# dataset = Dataset.from_pandas(test[['text']])

# def train_corp_iter():
#     for i in range(0, len(dataset), 1000):
#         yield dataset[i: i + 1000]["text"]

# raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

# tokenizer = PreTrainedTokenizerFast(
#     tokenizer_object=raw_tokenizer,
#     unk_token="[UNK]",
#     pad_token="[PAD]",
#     cls_token="[CLS]",
#     sep_token="[SEP]",
#     mask_token="[MASK]",
#     bos_token="[BOS]"
# )

# tokenized_texts_test = [tokenizer.tokenize(text) for text in tqdm(test['text'].tolist())]
# tokenized_texts_train = [tokenizer.tokenize(text) for text in tqdm(train['text'].tolist())]

In [17]:
# vectorizer = TfidfVectorizer(ngram_range=(3, 5),tokenizer=lambda x: re.findall(r'[^\W]+', x),token_pattern=None,strip_accents='unicode',)
# vectorizer = vectorizer.fit([" ".join(text) for text in tokenized_texts_train])
X = load_npz('data/train_tfidf_embeddings.npz')

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X[:train.shape[0]], train.target, test_size=0.2, random_state=42)

In [21]:
lr = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)

sgd_model = SGDClassifier(max_iter=4096, tol=1e-3, loss="modified_huber", random_state=42)
sgd_model2 = SGDClassifier(max_iter=4096, tol=1e-4, loss="modified_huber", class_weight="balanced", random_state=42)
sgd_model3 = SGDClassifier(max_iter=4096, tol=5e-4, loss="modified_huber", early_stopping=True, random_state=42)

mnb_model = MultinomialNB(alpha=0.02)
mnb_model2 = MultinomialNB(alpha=0.1)
mnb_model3 = MultinomialNB(alpha=0.2)

catboost_model = CatBoostClassifier(
    iterations=512, learning_rate=0.1, depth=3,
    loss_function='MultiClass', eval_metric='MultiClass', random_state=42
)

In [ ]:
ensemble = VotingClassifier(estimators=[
        ('lr', lr),
        ('sgd', sgd_model),
        ('sgd2', sgd_model2),
        ('sgd3', sgd_model3),
        ('cat', catboost_model),
        ('mnb', mnb_model),
        ('mnb2', mnb_model2),
        ('mnb3', mnb_model3),
    ],voting='soft')
ensemble.fit(X_train, y_train)

d:\cs\envs\dpl\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.metrics import accuracy_score
preds_test_classes = ensemble.predict(X_test)

accuracy = accuracy_score(y_test, preds_test_classes)

print(f'Accuracy on the test set: {accuracy:.4f}')


Accuracy on the test set: 0.6989


In [ ]:
test_X = load_npz('data/test_tfidf_embeddings.npz')

In [13]:
preds_test = ensemble.predict(test_X)

In [14]:
preds_test

array(['depression', 'depression', 'depression', ...,
       'relationship-and-family-issues', 'relationship-and-family-issues',
       'anxiety'], dtype=object)

In [15]:
pd.DataFrame({'id':test["id"],'target':preds_test}).sort_values(by="id").to_csv('submission_ensemble_bert.csv', index=False)